# Introduction

The provided notebook is a Python program for cryptocurrency data analysis and portfolio optimization. It uses historical price data and market cap data for selected cryptocurrencies to perform portfolio optimization using the Efficient Frontier approach. The script aims to calculate the profit/loss of the portfolio over different time periods and with varying numbers of coins.

# Notebook
## Import libs

In [1]:
import sys
sys.path.append("../")

import itertools
from time import time

import numpy as np
import pandas as pd


from src import Crypto_class as cc

## Functions

In [2]:
def calculate_profit(df, _top_100, _n_coins, _mu_method, _cov_method, _obj_function, _budget, _n_days_vector, _drop,
                     _hodl, _save_dir):

    crypto_class = cc.Cryptos(top_100, _budget, _n_coins, _hodl, _save_dir)
    results = {}
    portf = {}

    for n_days in _n_days_vector:
        crypto_class.validate_from_past(_n_coins, n_days, _mu_method, _cov_method, _obj_function, _drop)
        date = df.iloc[n_days].name

        results[date] = crypto_class.p_l
        portf[date] = crypto_class.portfolio_from_past

    p_l = sum(results.values())

    return p_l, results, portf

def check_coins(portfolio):
    # Convert the dictionary to a list of DataFrames
    dfs = [pd.DataFrame(portfolio[date]) for date in portfolio]

    # Concatenate the list of DataFrames into a single DataFrame
    result_df = pd.concat(dfs, keys=portfolio.keys())

    def get_total_coins(data):
        print(data["Coin"].iloc[0], data["n_coins"].sum())

    result_df.groupby("Coin").apply(get_total_coins)

## Inputs

In [3]:
start = time()

top_100 = False
drop = False
hodl = True

save_dir = './legacy_data'
n_years = 6

n_days_vector = np.arange(int(365 * n_years), 0, -30)
df_mc = pd.read_csv('./legacy_data/All_cryptos_market_cap.csv', index_col=0)

## Optimization parameters

In [4]:
possible_mu_methods = ["mean", "exp", "capm"]
possible_cov_methods = ["sample", "exp"]
possible_obj_functions = ["quadratic", "sharpe", "min_volat"]

# Get all combinations of mu_method, cov_method, and obj_function
all_combinations = list(itertools.product(possible_mu_methods, possible_cov_methods, possible_obj_functions))

## Variables initialization

In [5]:
# you place 100e each month per category for the past n years
budget = 100

maxpl2 = 0
maxpl10 = 0
maxpl20 = 0


best_portf_2 = None
best_portf_10 = None
best_portf_20 = None

best_res_2 = None
best_res_10 = None 
best_res_20 = None  

optimized_variables = dict()

## Run Analysis

In [6]:
# Test the portfolio_optimization function with all combinations
start = time()
for mu_method, cov_method, obj_function in all_combinations:

    n_coins = 2
    print(f'\n===== {n_coins} COINS ======\n')
    
    try:
        p_l_2, results_2, portf_2 = calculate_profit(df_mc, top_100, n_coins, mu_method, cov_method,
                                                        obj_function, budget, n_days_vector, drop, hodl, save_dir)
    except Exception as e:
        pass
    if p_l_2 >= maxpl2:
        maxpl2 = p_l_2
        best_portf_2 = portf_2
        best_res2 = results_2
        
        optimized_variables["2_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function}
        
    n_coins = 10
    print(f'\n===== {n_coins} COINS ======\n')
    
    try:
        p_l_10, results_10, portf_10 = calculate_profit(df_mc, top_100, n_coins, mu_method, cov_method,
                                                        obj_function, budget, n_days_vector, drop, hodl, save_dir)
    except Exception as e:
        pass
    if p_l_10 >= maxpl10:
        maxpl10 = p_l_10
        best_portf_10 = portf_10
        best_res10 = results_10
        
        optimized_variables["10_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function}
            
    n_coins = 20

    print(f'\n===== {n_coins} COINS ======\n')

    try:
        p_l_20, results_20, portf_20 = calculate_profit(df_mc, top_100, n_coins, mu_method, cov_method,
                                                        obj_function, budget, n_days_vector, drop, hodl, save_dir)
    except Exception as e:
        pass
    
    if p_l_20 >= maxpl20:
        maxpl20 = p_l_20
        best_portf_20 = portf_20
        best_res20 = results_20
        
        optimized_variables["20_coins"] = {
            "mu": mu_method,
            "s": cov_method,
            "obj_function": obj_function}
        
        

end = time()
print(f'Time elapsed: {(end - start) / 60}')


===== 2 COINS ======

Expected annual return: 757.1%
Annual volatility: 159.9%
Sharpe Ratio: 4.72
Invest 100.0

 Profit Loss: 863.3017353819275e

Expected annual return: 947.1%
Annual volatility: 157.6%
Sharpe Ratio: 5.99
Invest 100.0

 Profit Loss: 434.74982028464933e

Expected annual return: 775.1%
Annual volatility: 157.5%
Sharpe Ratio: 4.91
Invest 100.0

 Profit Loss: 547.2203733851914e

Expected annual return: 719.0%
Annual volatility: 155.0%
Sharpe Ratio: 4.63
Invest 100.0

 Profit Loss: 527.382049681436e

Expected annual return: 824.8%
Annual volatility: 153.0%
Sharpe Ratio: 5.38
Invest 100.0

 Profit Loss: 298.98198000936793e

Expected annual return: 954.5%
Annual volatility: 152.9%
Sharpe Ratio: 6.23
Invest 100.0

 Profit Loss: 142.91912594124867e

Expected annual return: 1011.7%
Annual volatility: 152.5%
Sharpe Ratio: 6.62
Invest 100.0

 Profit Loss: 75.70762287991333e

Expected annual return: 849.9%
Annual volatility: 152.2%
Sharpe Ratio: 5.57
Invest 100.0

 Profit Loss: 11

Expected annual return: 117.7%
Annual volatility: 99.4%
Sharpe Ratio: 1.16
Invest 100.0

 Profit Loss: 20.42523991827651e

Expected annual return: 117.0%
Annual volatility: 99.1%
Sharpe Ratio: 1.16
Invest 100.0

 Profit Loss: 16.387115755595687e

Expected annual return: 117.5%
Annual volatility: 98.1%
Sharpe Ratio: 1.18
Invest 100.0

 Profit Loss: 3.5724787325422267e

Expected annual return: 117.3%
Annual volatility: 97.9%
Sharpe Ratio: 1.18
Invest 100.0

 Profit Loss: -1.3792926749891496e

Expected annual return: 115.8%
Annual volatility: 97.7%
Sharpe Ratio: 1.17
Invest 100.0

 Profit Loss: -0.9781093366100397e

Expected annual return: 112.8%
Annual volatility: 95.8%
Sharpe Ratio: 1.16
Invest 100.0

 Profit Loss: -2.4508485151556743e


===== 10 COINS ======

Expected annual return: 1539.7%
Annual volatility: 171.8%
Sharpe Ratio: 8.95
Invest 100.0

 Profit Loss: -72.46192237193004e

Expected annual return: 8306.5%
Annual volatility: 279.3%
Sharpe Ratio: 29.73
Invest 100.0

 Profit Loss

/Users/dimitrisglenis/anaconda3/lib/python3.8/site-packages/pypfopt/risk_models.py:68: UserWarning: The covariance matrix is non positive semidefinite. Amending eigenvalues.
  warnings.warn(
/Users/dimitrisglenis/anaconda3/lib/python3.8/site-packages/pypfopt/risk_models.py:87: UserWarning: Could not fix matrix. Please try a different risk model.
  warnings.warn(


Expected annual return: 3878.3%
Annual volatility: 278.7%
Sharpe Ratio: 13.91
Invest 100.0

 Profit Loss: -66.84252404916899e

Expected annual return: 3141.0%
Annual volatility: 271.3%
Sharpe Ratio: 11.57
Invest 100.0

 Profit Loss: -69.12641148418282e

Expected annual return: 4846.5%
Annual volatility: 260.7%
Sharpe Ratio: 18.59
Invest 100.0

 Profit Loss: -84.42917939949139e

Expected annual return: 17687921.3%
Annual volatility: 424.2%
Sharpe Ratio: 41701.03
Invest 100.0

 Profit Loss: -26.1400578780724e

Expected annual return: 2706669.7%
Annual volatility: 387.1%
Sharpe Ratio: 6991.69
Invest 100.0

 Profit Loss: -50.57513374299726e

Expected annual return: 74288.0%
Annual volatility: 354.3%
Sharpe Ratio: 209.65
Invest 100.0

 Profit Loss: -10.928791316962217e

Expected annual return: 6388.4%
Annual volatility: 329.4%
Sharpe Ratio: 19.39
Invest 100.0

 Profit Loss: 66.58566531569224e

Expected annual return: 8913.8%
Annual volatility: 309.8%
Sharpe Ratio: 28.76
Invest 100.0

 Profi

Expected annual return: 322.6%
Annual volatility: 124.0%
Sharpe Ratio: 2.59
Invest 100.0

 Profit Loss: -29.508521431770987e

Expected annual return: 309.3%
Annual volatility: 122.6%
Sharpe Ratio: 2.51
Invest 100.0

 Profit Loss: -27.71270222825983e

Expected annual return: 298.6%
Annual volatility: 122.0%
Sharpe Ratio: 2.43
Invest 100.0

 Profit Loss: -26.53503461594614e

Expected annual return: 287.5%
Annual volatility: 121.4%
Sharpe Ratio: 2.35
Invest 100.0

 Profit Loss: -23.5870631713887e

Expected annual return: 267.7%
Annual volatility: 131.2%
Sharpe Ratio: 2.02
Invest 100.0

 Profit Loss: 1.2657701737880527e


===== 20 COINS ======

Expected annual return: 1608.7%
Annual volatility: 166.2%
Sharpe Ratio: 9.67
Invest 100.0

 Profit Loss: -38.41632672488966e



...

Expected annual return: 272.2%
Annual volatility: 80.0%
Sharpe Ratio: 3.38
Invest 100.0

 Profit Loss: -64.19342919275866e

Expected annual return: 255.8%
Annual volatility: 95.4%
Sharpe Ratio: 2.66
Invest 100.00000000000001

 Profit Loss: -73.3506996347588e

Expected annual return: 313.2%
Annual volatility: 119.2%
Sharpe Ratio: 2.61
Invest 100.0

 Profit Loss: -58.112656814136656e

Expected annual return: 353.1%
Annual volatility: 141.7%
Sharpe Ratio: 2.48
Invest 100.0

 Profit Loss: -69.80807812472462e

Expected annual return: 336.7%
Annual volatility: 122.2%
Sharpe Ratio: 2.74
Invest 100.0

 Profit Loss: -68.33259032215918e

Expected annual return: 344.1%
Annual volatility: 112.5%
Sharpe Ratio: 3.04
Invest 100.0

 Profit Loss: -75.4098818495956e

Expected annual return: 331.0%
Annual volatility: 108.7%
Sharpe Ratio: 3.03
Invest 100.0

 Profit Loss: -70.79547418989124e

Expected annual return: 333.5%
Annual volatility: 96.5%
Sharpe Ratio: 3.43
Invest 100.0

 Profit Loss: -72.31581988

Expected annual return: 270.3%
Annual volatility: 73.1%
Sharpe Ratio: 3.67
Invest 100.0

 Profit Loss: 180.96385083140353e

Expected annual return: 263.7%
Annual volatility: 71.3%
Sharpe Ratio: 3.67
Invest 100.0

 Profit Loss: 161.93087386043567e

Expected annual return: 271.3%
Annual volatility: 72.8%
Sharpe Ratio: 3.70
Invest 100.0

 Profit Loss: 126.28447929408672e

Expected annual return: 264.0%
Annual volatility: 65.3%
Sharpe Ratio: 4.01
Invest 100.00000000000001

 Profit Loss: 117.654129186662e

Expected annual return: 252.3%
Annual volatility: 56.9%
Sharpe Ratio: 4.40
Invest 100.0

 Profit Loss: 130.6150490336908e

Expected annual return: 285.7%
Annual volatility: 70.4%
Sharpe Ratio: 4.03
Invest 100.0

 Profit Loss: 79.63156456583036e

Expected annual return: 265.9%
Annual volatility: 65.0%
Sharpe Ratio: 4.06
Invest 100.0

 Profit Loss: 132.69139388643976e

Expected annual return: 5233.0%
Annual volatility: 496.2%
Sharpe Ratio: 10.54
Invest 100.0

 Profit Loss: -43.6554849898057

Expected annual return: 107.7%
Annual volatility: 62.9%
Sharpe Ratio: 1.68
Invest 99.99999999999999

 Profit Loss: 525.4082287432543e

Expected annual return: 99.1%
Annual volatility: 63.4%
Sharpe Ratio: 1.53
Invest 100.0

 Profit Loss: 273.40319644528984e

Expected annual return: 102.1%
Annual volatility: 57.7%
Sharpe Ratio: 1.73
Invest 100.0

 Profit Loss: 185.97615137425367e

Expected annual return: 83.1%
Annual volatility: 97.7%
Sharpe Ratio: 0.83
Invest 100.0

 Profit Loss: 461.70446070309197e

Expected annual return: 88.0%
Annual volatility: 97.1%
Sharpe Ratio: 0.89
Invest 100.0

 Profit Loss: 326.74256258464595e

Expected annual return: 93.2%
Annual volatility: 91.7%
Sharpe Ratio: 1.00
Invest 100.0

 Profit Loss: 215.11033460364794e

Expected annual return: 94.2%
Annual volatility: 83.6%
Sharpe Ratio: 1.10
Invest 100.0

 Profit Loss: 208.10221037397295e

Expected annual return: 92.6%
Annual volatility: 72.2%
Sharpe Ratio: 1.25
Invest 100.0

 Profit Loss: 214.88912573820144e

Exp

Expected annual return: 189.8%
Annual volatility: 65.6%
Sharpe Ratio: 2.86
Invest 100.00000000000001

 Profit Loss: 285.666984179572e

Expected annual return: 223.6%
Annual volatility: 77.0%
Sharpe Ratio: 2.88
Invest 99.99999999999999

 Profit Loss: 369.5838377965391e

Expected annual return: 218.4%
Annual volatility: 71.1%
Sharpe Ratio: 3.04
Invest 100.0

 Profit Loss: 395.72408510442904e

Expected annual return: 209.5%
Annual volatility: 64.3%
Sharpe Ratio: 3.23
Invest 100.0

 Profit Loss: 444.2178308130799e

Expected annual return: 171.9%
Annual volatility: 64.3%
Sharpe Ratio: 2.64
Invest 100.0

 Profit Loss: 378.724828693714e

Expected annual return: 183.9%
Annual volatility: 62.7%
Sharpe Ratio: 2.90
Invest 100.0

 Profit Loss: 359.4558501316245e

Expected annual return: 160.9%
Annual volatility: 57.7%
Sharpe Ratio: 2.76
Invest 100.0

 Profit Loss: 313.11437987403895e

Expected annual return: 171.9%
Annual volatility: 60.7%
Sharpe Ratio: 2.80
Invest 100.0

 Profit Loss: 271.9096283

Expected annual return: 243.3%
Annual volatility: 59.6%
Sharpe Ratio: 4.05
Invest 100.00000000000001

 Profit Loss: 312.12031895398445e

Expected annual return: 228.5%
Annual volatility: 64.5%
Sharpe Ratio: 3.51
Invest 100.0

 Profit Loss: 548.7650814176277e

Expected annual return: 214.1%
Annual volatility: 75.2%
Sharpe Ratio: 2.82
Invest 100.0

 Profit Loss: 595.6724246516364e

Expected annual return: 239.0%
Annual volatility: 72.7%
Sharpe Ratio: 3.26
Invest 100.0

 Profit Loss: 683.4290917684641e

Expected annual return: 214.7%
Annual volatility: 65.5%
Sharpe Ratio: 3.25
Invest 100.0

 Profit Loss: 645.9861917011847e

Expected annual return: 232.3%
Annual volatility: 58.6%
Sharpe Ratio: 3.93
Invest 100.0

 Profit Loss: 599.0648449621609e

Expected annual return: 252.6%
Annual volatility: 60.7%
Sharpe Ratio: 4.13
Invest 99.99999999999997

 Profit Loss: 434.27210359924817e

Expected annual return: 283.7%
Annual volatility: 69.2%
Sharpe Ratio: 4.07
Invest 99.99999999999999

 Profit Los

In [7]:
check_coins(best_portf_20)
print(maxpl20)
optimized_variables["20_coins"]

AAVE 0.44430037855204413
ADA 240.17503906713017
BNB 418.4950039068324
DOT 31.345311148703857
ETC 0.0640419151813918
ICX 11.99570085107819
MATIC 24.037822998623472
NEO 10.40885278525
SOL 17.93488304843077
UNI 2.4748133187556993
XEM 410.13065320521855
94206.29952872476


{'mu': 'mean', 's': 'exp', 'obj_function': 'quadratic'}

In [8]:
check_coins(best_portf_2)
print(maxpl2)
optimized_variables["2_coins"]

BTC 0.05788575861267857
ETH 16.783171993985988
XRP 180.63356338226913
25742.486503175922


{'mu': 'mean', 's': 'exp', 'obj_function': 'sharpe'}

In [9]:
check_coins(best_portf_10)
print(maxpl10)
optimized_variables["10_coins"]

ADA 2161.4970579898636
BCH 0.06364278498092425
BNB 121.77148370439932
DOT 43.53180480720994
EOS 2.6249035321087475
ETC 1.7870306850340072
ETH 1.9633971878649725
MATIC 24.037822998623472
NEO 9.815609340912538
SOL 10.05335732372861
TRX 13454.178704424276
UNI 11.363474598013081
XEM 450.55918778848013
28127.22626854699


{'mu': 'mean', 's': 'exp', 'obj_function': 'quadratic'}

## Results

In [10]:
print(f'Profit 2c: {round(maxpl2, 2)} $')
print(f'Profit 10c: {round(maxpl10, 2)} $')
print(f'Profit 20c: {round(maxpl20, 2)} $')
# print(f'Profit 100c: {round(maxpl100, 2)} $')

print(f'\nInvestment: {3 * budget * len(n_days_vector)} $')
print(f'Final Profit: {round(maxpl2 + maxpl10 + maxpl20 , 2)} $') # + maxpl100
print(f'Final Profit: {round((maxpl2 + maxpl10 + maxpl20 ) / (3 * budget * len(n_days_vector)) * 100, 2)} %\n') # + maxpl100

end = time()
print(f'Time elapsed: {(end - start) / 60}')

Profit 2c: 25742.49 $
Profit 10c: 28127.23 $
Profit 20c: 94206.3 $

Investment: 21900 $
Final Profit: 148076.01 $
Final Profit: 676.15 %

Time elapsed: 23.877577296892802


# Conclusion

## 2 Coins

In [11]:
optimized_variables["2_coins"]

{'mu': 'mean', 's': 'exp', 'obj_function': 'sharpe'}

## 10 Coins

In [12]:
optimized_variables["10_coins"]

{'mu': 'mean', 's': 'exp', 'obj_function': 'quadratic'}

## 20 Coins

In [13]:
optimized_variables["20_coins"]

{'mu': 'mean', 's': 'exp', 'obj_function': 'quadratic'}